In [1]:
import polars as pl
import os

In [2]:
cynde_base = r"C:\Users\Tommaso\Documents\Dev\Cynde"
cynde_data = os.path.join(cynde_base, "data_processed")

In [3]:
joined_predictions = "joined_2024-03-06_11-34-46_test.parquet"
joined_predictions_url = os.path.join(cynde_data, joined_predictions)
pred_dfs = "predictions_2024-03-06_11-34-46_test.parquet"
pred_dfs_url = os.path.join(cynde_data, pred_dfs)




In [4]:
print(joined_predictions_url)

C:\Users\Tommaso\Documents\Dev\Cynde\data_processed\joined_2024-03-06_11-34-46_test.parquet


In [5]:
df = pl.read_parquet(joined_predictions_url)
pred_df = pl.read_parquet(pred_dfs_url)
print(df.shape)
print(pred_df.shape)

(5824, 2825)
(5824, 2822)


In [6]:
from cynde.functional.results import get_predictions

In [7]:
models_dict = {"RandomForest": [{"n_estimators": 10, "max_depth": 5},{"n_estimators": 50, "max_depth": 10}]}
inputs =[{"numerical":["conversations_text-embedding-3-small_embedding"]},
            {"numerical":["conversations_text-embedding-3-large_embedding"]},
            {"numerical":["conversations_text-embedding-3-small_embedding","conversations_text-embedding-3-large_embedding"]}]

In [8]:
r_out = 10
k_out = 2
r_in = 10
k_in = 2
cv_type = ("stratified","stratified")

In [9]:
pred_cols = [col for col in pred_df.columns if "pred" in col]

In [10]:
pred_cols

['outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_0_conversations_text-embedding-3-small_embedding_RandomForest_n_estimators_10_max_depth_5_y_pred',
 'outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_0_conversations_text-embedding-3-small_embedding_RandomForest_n_estimators_50_max_depth_10_y_pred',
 'outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_0_conversations_text-embedding-3-large_embedding_RandomForest_n_estimators_10_max_depth_5_y_pred',
 'outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_0_conversations_text-embedding-3-large_embedding_RandomForest_n_estimators_50_max_depth_10_y_pred',
 'outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_0_conversations_text-embedding-3-small_embedding_conversations_text-embedding-3-large_embedding_RandomForest_n_estimators_10_max_depth_5_y_pred',
 'outer_stratified_target_replica_0_fold_0_inn

In [11]:
outs = get_predictions(df,
                       cv_type=cv_type,
                          models=models_dict,
                          group_outer=["target"],
                            inputs=inputs,
                            r_outer=r_out,
                            k_outer=k_out,
                            group_inner=["target"],
                            r_inner=r_in,
                            k_inner=k_in,)                 

In [17]:
df

cv_index,conversations,source,target,target_right,outer_stratified_target_replica_0_fold_0,outer_stratified_target_replica_0_fold_1,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_0,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_1,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_1_fold_0,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_1_fold_1,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_2_fold_0,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_2_fold_1,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_3_fold_0,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_3_fold_1,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_4_fold_0,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_4_fold_1,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_5_fold_0,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_5_fold_1,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_6_fold_0,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_6_fold_1,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_7_fold_0,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_7_fold_1,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_8_fold_0,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_8_fold_1,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_9_fold_0,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_9_fold_1,outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_0_fold_0,outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_0_fold_1,outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_1_fold_0,outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_1_fold_1,outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_2_fold_0,outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_2_fold_1,outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_3_fold_0,outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_3_fold_1,outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_4_fold_0,outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_4_fold_1,…,outer_stratified_target_replica_9_fold_1_inner_stratified_target_replica_6_fold_1_conversations_text-embedding-3-small_embedding_conversations_text-embedding-3-large_embedding_RandomForest_n_estimators_50_max_depth_10_y_pred,outer_stratified_target_replica_9_fold_1_inner_stratified_target_replica_7_fold_0_conversations_text-embedding-3-small_embedding_RandomForest_n_estimators_10_max_depth_5_y_pred,outer_stratified_target_replica_9_fold_1_inner_stratified_target_replica_7_fold_0_conversations_text-embedding-3-small_embedding_RandomForest_n_estimators_50_max_depth_10_y_pred,outer_stratified_target_replica_9_fold_1_inner_stratified_target_replica_7_fold_0_conversations_text-embedding-3-large_embedding_RandomForest_n_estimators_10_max_depth_5_y_pred,outer_stratified_target_replica_9_fold_1_inner_stratified_target_replica_7_fold_0_conversations_text-embedding-3-large_embedding_RandomForest_n_estimators_50_max_depth_10_y_pred,outer_stratified_target_replica_9_fold_1_inner_stratified_target_replica_7_fold_0_conversations_text-embedding-3-small_embedding_conversations_text-embedding-3-large_embedding_RandomForest_n_estimators_10_max_depth_5_y_pred,outer_stratified_target_replica_9_fold_1_inner_stratified_target_replica_7_fold_0_conversations_text-embedding-3-small_embedding_conversations_text-embedding-3-large_embedding_RandomForest_n_estimators_50_max_depth_10_y_pred,outer_stratified_targe

In [16]:
df.select(pl.col(["cv_index","target",outs[0][0],outs[0][1]]))

cv_index,target,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_0,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_0_conversations_text-embedding-3-small_embedding_RandomForest_n_estimators_10_max_depth_5_y_pred
u32,bool,str,bool
0,true,"""train""",true
1,true,"""test""",false
2,true,"""train""",true
3,true,"""test""",false
4,true,"""test""",false
5,true,"""train""",true
6,true,"""test""",false
7,true,"""val""",false
8,true,"""test""",false


In [14]:
outs[0]

('outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_0',
 'outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_0_conversations_text-embedding-3-small_embedding_RandomForest_n_estimators_10_max_depth_5_y_pred')

In [12]:
pred_cols[0]

'outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_0_conversations_text-embedding-3-small_embedding_RandomForest_n_estimators_10_max_depth_5_y_pred'

In [13]:
outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_0_RandomForest_n_estimators_10_max_depth_5_conversations_text-embedding-3-small_embedding_y_pred
outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_0_conversations_text-embedding-3-small_embedding_RandomForest_n_estimators_10_max_depth_5_y_pred

NameError: name 'outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_0_RandomForest_n_estimators_10_max_depth_5_conversations_text' is not defined